In [6]:
!pip install flask_ngrok
!pip install pyngrok
!ngrok config add-authtoken 33TFiQQQT1Oe1j0enzr4eid1j9b_jT4fD2Jem5oWAUd1Q4Dn

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
%run /content/drive/MyDrive/project/blood_donation_app.ipynb

Exception: File `'/content/drive/MyDrive/project/blood_donation_app.ipynb.py'` not found.

In [8]:
# -*- coding: utf-8 -*-
"""Blood Donation Flask App

Adapted from Tkinter GUI to Flask web app.
Original Tkinter code fixes applied:
- Fixed table name consistency (Sign_Up).
- Fixed Confirm_Password_Edit.get() in signup.
- Added creation of Verification table with default values.
- Fixed Haemoglobin as REAL in Donors table.
- Handled potential errors in inserts.
- Globals removed; data passed via session.
- Enhanced ngrok setup for reliable public URL access in Colab.
- Fixed typo in signup route (basis_auth -> render_template_string).
- Added test endpoint for debugging.
"""

import subprocess
import sys
import time
import os
from google.colab import drive
from flask import Flask, request, jsonify, render_template_string, session, redirect, url_for
from pyngrok import ngrok
import sqlite3

# Install required packages quietly
try:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'flask', 'pyngrok'])
    print("Successfully installed flask and pyngrok")
except Exception as e:
    print(f"Error installing packages: {e}")
    raise

# Mount Google Drive
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    raise

# Navigate to your project folder in Drive for persistence
project_path = '/content/drive/MyDrive/project'
os.makedirs(project_path, exist_ok=True)
os.chdir(project_path)

# Verify you're in the right location
print("Current working directory:", os.getcwd())
try:
    ls_output = subprocess.check_output(['ls', '-la']).decode()
    print("Folder contents:\n" + ls_output)
except Exception as e:
    print(f"Error listing directory contents: {e}")

app = Flask(__name__)
app.secret_key = 'blood_donation_secret_key'  # For session management

# Setup ngrok authtoken
NGROK_TOKEN = "33TFiQQQT1Oe1j0enzr4eid1j9b_jT4fD2Jem5oWAUd1Q4Dn"
try:
    subprocess.check_call(['ngrok', 'config', 'add-authtoken', NGROK_TOKEN])
    print("Ngrok authtoken configured successfully")
except Exception as e:
    print(f"Error configuring ngrok authtoken: {e}")
    raise

def init_db():
    db_path = os.path.join(os.getcwd(), 'BloodDonation.db')
    try:
        conn = sqlite3.connect(db_path)
        c = conn.cursor()

        # Users table (fixed name consistency)
        c.execute('''CREATE TABLE IF NOT EXISTS Sign_Up
                     (First_Name TEXT NOT NULL, Last_Name TEXT NOT NULL,
                      Email TEXT NOT NULL UNIQUE, Password TEXT NOT NULL,
                      Confim_Password TEXT NOT NULL)''')  # Added UNIQUE for Email

        # Verification table (added with defaults)
        c.execute('''CREATE TABLE IF NOT EXISTS Verification
                     (Min_Weight INTEGER, Max_Weight INTEGER,
                      Min_Age INTEGER, Max_Age INTEGER,
                      Min_Haemoglobin REAL, Max_Haemoglobin REAL,
                      Min_Donation_Interval INTEGER, Max_Donation_Interval INTEGER,
                      Q1 TEXT, Q2 TEXT, Q3 TEXT)''')

        # Insert defaults if empty
        c.execute("SELECT COUNT(*) FROM Verification")
        if c.fetchone()[0] == 0:
            c.execute("INSERT INTO Verification VALUES (50, 100, 17, 60, 12.5, 17.0, 0, 52, 'Good', 'No', 'No')")

        # Donors table (Haemoglobin as REAL)
        c.execute('''CREATE TABLE IF NOT EXISTS Donors
                     (First_Name TEXT, Last_Name TEXT, Blood_Group TEXT,
                      Age INTEGER, Weight INTEGER, Donation_Interval INTEGER,
                      Haemoglobin REAL, Gender TEXT, Question_1 TEXT,
                      Question_2 TEXT, Question_3 TEXT)''')

        conn.commit()
        conn.close()
        print("Database initialized successfully")
    except Exception as e:
        print(f"Error initializing database: {e}")
        raise

def get_db_connection():
    db_path = os.path.join(os.getcwd(), 'BloodDonation.db')
    return sqlite3.connect(db_path)

# Test endpoint to verify server
@app.route('/test')
def test():
    return "Flask server is running!"

@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')
        if not username or not password:
            return render_template_string(login_template, error="Missing credentials")

        conn = get_db_connection()
        c = conn.cursor()
        c.execute('SELECT * FROM Sign_Up WHERE Email = ? AND Password = ?', (username, password))
        user = c.fetchone()
        conn.close()

        if user:
            session['logged_in'] = True
            session['user_email'] = username
            session['user_first'] = user[0]
            session['user_last'] = user[1]
            return redirect(url_for('eligibility'))
        else:
            return render_template_string(login_template, error="Invalid username or password")

    return render_template_string(login_template, error=None)

login_template = """
<!DOCTYPE html>
<html>
<head><title>Login</title></head>
<body style="background-color: #59ABE3;">
<h1 style="text-align: center; background-color: #264348; color: white; padding: 10px;">Blood Donation System - Login</h1>
<div style="background-color: #d9d9da; width: 400px; margin: auto; padding: 20px;">
{% if error %}
    <p style="color: red;">{{ error }}</p>
{% endif %}
<form method="post">
    <label>Username (Email):</label><br>
    <input type="text" name="username" required><br><br>
    <label>Password:</label><br>
    <input type="password" name="password" required><br><br>
    <input type="submit" value="Login" style="background-color: #5B3256; color: white; padding: 5px;">
</form>
<br>
<a href="{{ url_for('signup') }}">Sign Up</a>
</div>
</body>
</html>
"""

@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        first_name = request.form.get('first_name')
        last_name = request.form.get('last_name')
        email = request.form.get('email')
        password = request.form.get('password')
        confirm_password = request.form.get('confirm_password')

        if not all([first_name, last_name, email, password, confirm_password]):
            return render_template_string(signup_template, error="All fields are required")

        if password != confirm_password:
            return render_template_string(signup_template, error="Passwords do not match")

        conn = get_db_connection()
        c = conn.cursor()
        try:
            c.execute("INSERT INTO Sign_Up VALUES (?, ?, ?, ?, ?)",
                      (first_name, last_name, email, password, confirm_password))
            conn.commit()
            conn.close()
            return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close()
            return render_template_string(signup_template, error="User already exists")
        except Exception as e:
            conn.close()
            return render_template_string(signup_template, error=f"Error: {str(e)}")

    return render_template_string(signup_template, error=None)

signup_template = """
<!DOCTYPE html>
<html>
<head><title>Sign Up</title></head>
<body style="background-color: #59ABE3;">
<h1 style="text-align: center; background-color: #264348; color: white; padding: 10px;">Sign Up</h1>
<div style="background-color: #d9d9da; width: 400px; margin: auto; padding: 20px;">
{% if error %}
    <p style="color: red;">{{ error }}</p>
{% endif %}
<form method="post">
    <label>First Name:</label><br>
    <input type="text" name="first_name" required><br><br>
    <label>Last Name:</label><br>
    <input type="text" name="last_name" required><br><br>
    <label>Email (Username):</label><br>
    <input type="email" name="email" required><br><br>
    <label>Password:</label><br>
    <input type="password" name="password" required><br><br>
    <label>Confirm Password:</label><br>
    <input type="password" name="confirm_password" required><br><br>
    <input type="submit" value="Sign Up" style="background-color: #5B3256; color: white; padding: 5px;">
</form>
<br>
<a href="{{ url_for('login') }}">Back to Login</a>
</div>
</body>
</html>
"""

@app.route('/eligibility', methods=['GET', 'POST'])
def eligibility():
    if not session.get('logged_in'):
        return redirect(url_for('login'))

    if request.method == 'POST':
        firstname = request.form.get('firstname')
        lastname = request.form.get('lastname')
        bloodgroup = request.form.get('bloodgroup')
        gender = request.form.get('gender')
        weight_str = request.form.get('weight')
        age_str = request.form.get('age')
        haemoglobin_str = request.form.get('haemoglobin')
        donation_interval_str = request.form.get('donation_interval')
        q1 = request.form.get('q1')
        q2 = request.form.get('q2')
        q3 = request.form.get('q3')

        try:
            weight = int(weight_str)
            age = int(age_str)
            haemoglobin = float(haemoglobin_str)
            donation_interval = int(donation_interval_str)
        except ValueError:
            return render_template_string(eligibility_template, error="Invalid numeric input")

        if not all([firstname, lastname, bloodgroup, gender, q1, q2, q3]):
            return render_template_string(eligibility_template, error="All fields are required")

        conn = get_db_connection()
        c = conn.cursor()
        c.execute("SELECT Min_Weight, Max_Weight, Min_Age, Max_Age, Min_Haemoglobin, Max_Haemoglobin, Min_Donation_Interval, Max_Donation_Interval, Q1, Q2, Q3 FROM Verification")
        ver = c.fetchone()
        conn.close()

        eligible = (
            ver[0] <= weight <= ver[1] and
            ver[2] <= age <= ver[3] and
            ver[4] <= haemoglobin <= ver[5] and
            ver[6] <= donation_interval <= ver[7] and
            q1 == ver[8] and
            q2 == ver[9] and
            q3 == ver[10]
        )

        if eligible:
            conn = get_db_connection()
            c = conn.cursor()
            c.execute("INSERT INTO Donors VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                      (firstname, lastname, bloodgroup, age, weight, donation_interval, haemoglobin, gender, q1, q2, q3))
            conn.commit()
            conn.close()

            # Store in session for report
            session['donor_data'] = {
                'firstname': firstname, 'lastname': lastname, 'bloodgroup': bloodgroup,
                'gender': gender, 'weight': weight, 'age': age, 'haemoglobin': haemoglobin,
                'donation_interval': donation_interval, 'q1': q1, 'q2': q2, 'q3': q3
            }
            return redirect(url_for('report'))
        else:
            return render_template_string(eligibility_template, error="Not eligible to donate blood. Please check your inputs.")

    return render_template_string(eligibility_template, error=None)

eligibility_template = """
<!DOCTYPE html>
<html>
<head><title>Eligibility</title></head>
<body style="background-color: #59ABE3;">
<h1 style="text-align: center; background-color: #264348; color: white; padding: 10px;">Eligibility Requirements</h1>
<div style="background-color: #d9d9da; width: 500px; margin: auto; padding: 20px;">
{% if error %}
    <p style="color: red;">{{ error }}</p>
{% endif %}
<form method="post">
    <label>First Name:</label><br><input type="text" name="firstname" required><br><br>
    <label>Last Name:</label><br><input type="text" name="lastname" required><br><br>
    <label>Blood Group:</label><br><input type="text" name="bloodgroup" required><br><br>
    <label>Gender (Male/Female):</label><br><input type="text" name="gender" required><br><br>
    <label>Weight (kg):</label><br><input type="number" name="weight" required><br><br>
    <label>Age (17-60):</label><br><input type="number" name="age" required><br><br>
    <label>Haemoglobin (g/L):</label><br><input type="number" step="0.1" name="haemoglobin" required><br><br>
    <label>Donation Interval (weeks):</label><br><input type="number" name="donation_interval" required><br><br>
    <label>General health? (Good/Bad):</label><br><input type="text" name="q1" required><br><br>
    <label>Fever in last 3 weeks? (Yes/No):</label><br><input type="text" name="q2" required><br><br>
    <label>Symptoms of infection? (Yes/No):</label><br><input type="text" name="q3" required><br><br>
    <input type="submit" value="Submit" style="background-color: #5B3256; color: white; padding: 10px;">
</form>
<br>
<a href="/blood_chart">Blood Distribution Chart</a><br>
<a href="/donors">View Donors List</a><br>
<a href="/logout">Logout</a>
</div>
</body>
</html>
"""

@app.route('/report')
def report():
    if not session.get('logged_in'):
        return redirect(url_for('login'))

    data = session.get('donor_data', {})
    if not data:
        return redirect(url_for('eligibility'))

    html = f"""
    <!DOCTYPE html>
    <html>
    <head><title>Report</title></head>
    <body style="background-color: #59ABE3;">
    <h1 style="text-align: center; background-color: #264348; color: white; padding: 10px;">Eligibility Report</h1>
    <div style="background-color: #d9d9da; width: 500px; margin: auto; padding: 20px;">
        <p><strong>Congratulations! You are eligible.</strong></p>
        <p>First Name: {data.get('firstname', '')}</p>
        <p>Last Name: {data.get('lastname', '')}</p>
        <p>Blood Group: {data.get('bloodgroup', '')}</p>
        <p>Gender: {data.get('gender', '')}</p>
        <p>Weight: {data.get('weight', '')} kg</p>
        <p>Age: {data.get('age', '')}</p>
        <p>Haemoglobin: {data.get('haemoglobin', '')} g/L</p>
        <p>Donation Interval: {data.get('donation_interval', '')} weeks</p>
        <p>Q1: {data.get('q1', '')}</p>
        <p>Q2: {data.get('q2', '')}</p>
        <p>Q3: {data.get('q3', '')}</p>
        <a href="/eligibility">Back to Form</a>
    </div>
    </body>
    </html>
    """
    return render_template_string(html)

@app.route('/blood_chart')
def blood_chart():
    if not session.get('logged_in'):
        return redirect(url_for('login'))

    labels = ['O+', 'A+', 'B+', 'AB+', 'O-', 'A-', 'B-', 'AB-']
    data = [38.67, 27.42, 22.02, 5.88, 2.55, 1.99, 1.11, 0.36]
    colors = ['#8F1D21', '#FFB3A7', '#F7CA18', '#1F4788', '#7A942E', '#049372', '#6C7A89', '#8D608C']

    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
    <title>Blood Distribution</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    </head>
    <body style="background-color: #59ABE3;">
    <h1 style="text-align: center; background-color: #264348; color: white; padding: 10px;">Blood Type Distribution in the World</h1>
    <canvas id="myChart" width="800" height="400" style="margin: auto; display: block;"></canvas>
    <script>
    const ctx = document.getElementById('myChart').getContext('2d');
    const myChart = new Chart(ctx, {{
        type: 'bar',
        data: {{
            labels: {labels},
            datasets: [{{
                label: 'Percentage (%)',
                data: {data},
                backgroundColor: {colors}
            }}]
        }},
        options: {{
            scales: {{
                y: {{
                    beginAtZero: true,
                    title: {{ display: true, text: 'Percentage (%)' }}
                }},
                x: {{
                    title: {{ display: true, text: 'Blood Group' }}
                }}
            }},
            plugins: {{
                title: {{ display: true, text: 'Blood Type Distribution' }}
            }}
        }}
    }});
    </script>
    <br><a href="/eligibility">Back</a>
    </body>
    </html>
    """
    return render_template_string(html)

@app.route('/donors')
def donors():
    if not session.get('logged_in'):
        return redirect(url_for('login'))

    conn = get_db_connection()
    c = conn.cursor()
    c.execute('SELECT * FROM Donors')
    rows = c.fetchall()
    conn.close()

    table_rows = ''.join(['<tr>' + ''.join([f'<td>{val}</td>' for val in row]) + '</tr>' for row in rows])

    html = f"""
    <!DOCTYPE html>
    <html>
    <head><title>Donors List</title></head>
    <body style="background-color: #59ABE3;">
    <h1 style="text-align: center; background-color: #264348; color: white; padding: 10px;">Donors List</h1>
    <div style="overflow-x: auto;">
    <table border="1" style="margin: auto; border-collapse: collapse;">
        <tr style="background-color: #264348; color: white;">
            <th>First Name</th><th>Last Name</th><th>Blood Group</th><th>Age</th><th>Weight</th>
            <th>Donation Interval</th><th>Haemoglobin</th><th>Gender</th><th>Q1</th><th>Q2</th><th>Q3</th>
        </tr>
        {table_rows}
    </table>
    </div>
    <br><a href="/eligibility">Back</a>
    </body>
    </html>
    """
    return render_template_string(html)

@app.route('/logout')
def logout():
    session.clear()
    return redirect(url_for('login'))

# API routes
@app.route('/api/hello')
def hello():
    name = request.args.get('name', 'World')
    return jsonify({"message": f"Hello, {name}!"})

@app.route('/api/info')
def info():
    return jsonify({
        "project": "Blood Donation Flask App",
        "location": "Google Colab with ngrok",
        "note": "Database persists in Drive folder"
    })

# Initialize DB before running
init_db()

# Start ngrok tunnel
try:
    print("🚀 Starting ngrok tunnel...")
    ngrok.kill()  # Ensure no stale tunnels
    public_url = ngrok.connect(5000, proto="http")
    print(f"🌐 Public URL: {public_url}")
    print("📱 Your app is now accessible! Copy the Public URL above and paste it into your browser. Do NOT use localhost or internal IPs like 127.0.0.1 or 172.28.0.12.")
except Exception as e:
    print(f"Error starting ngrok tunnel: {e}")
    raise

# Run Flask app in a separate thread to avoid blocking
import threading
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

if __name__ == '__main__':
    threading.Thread(target=run_flask).start()
    print("Flask app started. Waiting for ngrok to stabilize...")
    time.sleep(5)  # Give ngrok time to establish
    print(f"✅ Open this URL in your browser: {public_url}")
    print("If the URL doesn't work, check for errors above or re-run the cell.")

Mounted at /content/drive
Current working directory: /content/drive/MyDrive/project
Folder contents:
total 48
-rw------- 1 root root 31848 Jan 27 15:53 blood_donation_app.ipynb
-rw------- 1 root root 16384 Oct  3 05:10 BloodDonation.db

Ngrok authtoken configured successfully
Database initialized successfully
🚀 Starting ngrok tunnel...
🌐 Public URL: NgrokTunnel: "https://unintromittive-heide-unslippery.ngrok-free.dev" -> "http://localhost:5000"
📱 Your app is now accessible! Copy the Public URL above and paste it into your browser. Do NOT use localhost or internal IPs like 127.0.0.1 or 172.28.0.12.
Flask app started. Waiting for ngrok to stabilize...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


✅ Open this URL in your browser: NgrokTunnel: "https://unintromittive-heide-unslippery.ngrok-free.dev" -> "http://localhost:5000"
If the URL doesn't work, check for errors above or re-run the cell.


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Navigate to your project folder
import os

# Define your project path
project_path = '/content/drive/MyDrive/project'

# Create the folder if it doesn't exist
os.makedirs(project_path, exist_ok=True)

# Change to your project directory
os.chdir(project_path)

# Verify you're in the right location
print("Current working directory:", os.getcwd())
print("Folder contents:")
!ls -la